# Telangana State Analysis - Digital Equity Index (DEI)

This notebook performs analysis of Aadhaar enrollment and update data for Telangana's **33 official districts**.

## Data Sources
- **State Data**: `telangana_enrollment.csv`, `telangana_demographic.csv`, `telangana_biometric.csv`
- **Extracted Data**: `telangna_dist_in_andhra.csv` (Records found in Andhra dataset)

## Official Telangana Districts (33)
Adilabad, Bhadradri Kothagudem, Hanumakonda, Hyderabad, Jagtial, Jangaon, Jayashankar Bhupalpally, Jogulamba Gadwal, Kamareddy, Karimnagar, Khammam, Komaram Bheem Asifabad, Mahabubabad, Mahabubnagar, Mancherial, Medak, Medchal–Malkajgiri, Mulugu, Nagarkurnool, Nalgonda, Narayanpet, Nirmal, Nizamabad, Peddapalli, Rajanna Sircilla, Rangareddy, Sangareddy, Siddipet, Suryapet, Vikarabad, Wanaparthy, Warangal, Yadadri Bhuvanagiri

In [ ]:
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

## 1. Data Loading

In [ ]:
BASE_PATH = r'c:\Users\Admin\OneDrive\Documents\AADHAR Hackathon\telangana'
DATA_PATH = os.path.join(BASE_PATH, 'data')

# Main State Files
enrol_df = pd.read_csv(os.path.join(DATA_PATH, 'telangana_enrollment.csv'))
demo_df = pd.read_csv(os.path.join(DATA_PATH, 'telangana_demographic.csv'))
bio_df = pd.read_csv(os.path.join(DATA_PATH, 'telangana_biometric.csv'))

# Extracted Data from Andhra
tg_extracted_path = os.path.join(DATA_PATH, 'telangna_dist_in_andhra.csv')
if os.path.exists(tg_extracted_path):
    tg_extracted_df = pd.read_csv(tg_extracted_path)
    print(f'Loaded {len(tg_extracted_df)} records from extracted file.')
else:
    tg_extracted_df = pd.DataFrame()
    print('Warning: Extracted Telangana file not found.')

print(f'Enrollment: {len(enrol_df):,} | Demographic: {len(demo_df):,} | Biometric: {len(bio_df):,}')

In [ ]:
# Merge Extracted Data into respective DataFrames
# Note: The extracted file is a concatenation of all 3 types with a 'source_file' column
if not tg_extracted_df.empty:
    # Split back by source
    ext_enrol = tg_extracted_df[tg_extracted_df['source_file'] == 'Enrollment'].drop(columns=['source_file'])
    ext_demo = tg_extracted_df[tg_extracted_df['source_file'] == 'Demographic'].drop(columns=['source_file'])
    ext_bio = tg_extracted_df[tg_extracted_df['source_file'] == 'Biometric'].drop(columns=['source_file'])
    
    # Append
    enrol_df = pd.concat([enrol_df, ext_enrol], ignore_index=True)
    demo_df = pd.concat([demo_df, ext_demo], ignore_index=True)
    bio_df = pd.concat([bio_df, ext_bio], ignore_index=True)
    
    print(f'Merged Counts -> Enrollment: {len(enrol_df):,} | Demographic: {len(demo_df):,} | Biometric: {len(bio_df):,}')

## 2. Data Cleaning - Comprehensive Mapping

In [ ]:
# Official 33 TG Districts (lowercase)
# Note: 'medchal–malkajgiri' uses en-dash
OFFICIAL_DISTRICTS = {
    'adilabad', 'bhadradri kothagudem', 'hanumakonda', 'hyderabad', 'jagtial',
    'jangaon', 'jayashankar bhupalpally', 'jogulamba gadwal', 'kamareddy',
    'karimnagar', 'khammam', 'komaram bheem asifabad', 'mahabubabad',
    'mahabubnagar', 'mancherial', 'medak', 'medchal–malkajgiri', 'mulugu',
    'nagarkurnool', 'nalgonda', 'narayanpet', 'nirmal', 'nizamabad',
    'peddapalli', 'rajanna sircilla', 'rangareddy', 'sangareddy', 'siddipet',
    'suryapet', 'vikarabad', 'wanaparthy', 'warangal', 'yadadri bhuvanagiri'
}

DISTRICT_CLEANUP_MAP = {
    # --- Warangal Splitting ---
    'warangal urban': 'hanumakonda',
    'warangal (urban)': 'hanumakonda',
    'warangal rural': 'warangal',
    'warangal (rural)': 'warangal',
    
    # --- Short Names / Typos ---
    'bhadradri': 'bhadradri kothagudem',
    'jayashankar': 'jayashankar bhupalpally',
    'jogulamba': 'jogulamba gadwal',
    'komaram bheem': 'komaram bheem asifabad',
    'komaram bheem asifaba': 'komaram bheem asifabad',
    'rajanna': 'rajanna sircilla',
    'yadadri': 'yadadri bhuvanagiri',
    'yadadri bhuvanagi': 'yadadri bhuvanagiri',
    
    # --- Formatting / Typo / Encoding Issues ---
    'jangaon.': 'jangaon',
    'jagitial': 'jagtial',
    'jangoan': 'jangaon',
    
    # Medchal Variants
    'medchal-malkajgiri': 'medchal–malkajgiri', 
    'medchal?malkajgiri': 'medchal–malkajgiri',
    'medchalâ\x88\x92malkajgiri': 'medchal–malkajgiri',
    'medchalâ\x80\x93malkajgiri': 'medchal–malkajgiri',
    'medchal−malkajgiri': 'medchal–malkajgiri', # U+2212 Minus Sign
    'medchal malkajgiri': 'medchal–malkajgiri',

    # Rangareddy Variants
    'k.v. rangareddy': 'rangareddy',
    'k.v.rangareddy': 'rangareddy',
    'ranga reddy': 'rangareddy',
    'rangareddi': 'rangareddy',
    
    'mahabub nagar': 'mahabubnagar',
    'mahbubnagar': 'mahabubnagar',
    'karim nagar': 'karimnagar'
}

def clean_district_name(name):
    if pd.isna(name):
        return None
    
    # Lowercase and strip
    cleaned = str(name).strip().lower()
    
    # Remove asterisk
    if cleaned.endswith(' *'):
        cleaned = cleaned[:-2].strip()
    if cleaned.endswith('*'):
        cleaned = cleaned[:-1].strip()
        
    # Remove trailing dot (common in this dataset)
    if cleaned.endswith('.'):
        cleaned = cleaned[:-1].strip()
    
    # Apply mapping
    if cleaned in DISTRICT_CLEANUP_MAP:
        cleaned = DISTRICT_CLEANUP_MAP[cleaned]
        
    return cleaned

print(f'Official districts: {len(OFFICIAL_DISTRICTS)}')
print(f'Cleanup mappings: {len(DISTRICT_CLEANUP_MAP)}')

In [ ]:
# Apply cleaning and VALIDATE check
for df in [enrol_df, demo_df, bio_df]:
    # 1. Clean
    df['district_clean'] = df['district'].apply(clean_district_name)
    
    # 2. Check for unknowns
    # Note: Using replace for hyphen check to handle encoding differences just for validation
    unknowns_mask = ~df['district_clean'].isin(OFFICIAL_DISTRICTS)
    if unknowns_mask.any():
        # Try fuzzy check on dash
        unknown_vals = df[unknowns_mask]['district_clean'].unique()
        normalized_officials = {d.replace('–', '-') for d in OFFICIAL_DISTRICTS}
        really_unknown = []
        for u in unknown_vals:
             if pd.notna(u) and u.replace('–', '-') not in normalized_officials:
                 really_unknown.append(u)
        
        if really_unknown:
            print(f'⚠️ CRITICAL: Unmapped districts found: {really_unknown}')
            
    # 3. Apply
    df.dropna(subset=['district_clean'], inplace=True)
    df['district'] = df['district_clean']
    df.drop(columns=['district_clean'], inplace=True)

    # 4. Dates
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['month'] = df['date'].dt.month

# Verify
all_cleaned = set(enrol_df['district'].unique()) | set(demo_df['district'].unique()) | set(bio_df['district'].unique())
print(f'\nFinal Districts ({len(all_cleaned)}):')
print(sorted(all_cleaned))

if len(all_cleaned) != 33:
    print(f'⚠️ Warning: Expected 33 districts, found {len(all_cleaned)}.')
else:
    print('✅ Exactly 33 districts found!')

## 3. Aggregation & Metrics

In [ ]:
# Aggregate
enrol_agg = enrol_df.groupby(['state', 'district', 'month'])[['age_0_5', 'age_5_17', 'age_18_greater']].sum().reset_index()
demo_agg = demo_df.groupby(['state', 'district', 'month'])[['demo_age_5_17', 'demo_age_17_']].sum().reset_index()
bio_agg = bio_df.groupby(['state', 'district', 'month'])[['bio_age_5_17', 'bio_age_17_']].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on=['state', 'district', 'month'], how='outer') \
                       .merge(bio_agg, on=['state', 'district', 'month'], how='outer')
combined_df.fillna(0, inplace=True)

# Core metrics
combined_df['E'] = combined_df['age_0_5'] + combined_df['age_5_17'] + combined_df['age_18_greater']
combined_df['DU'] = combined_df['demo_age_5_17'] + combined_df['demo_age_17_']
combined_df['BU'] = combined_df['bio_age_5_17'] + combined_df['bio_age_17_']
combined_df['U'] = combined_df['DU'] + combined_df['BU']
combined_df['T'] = combined_df['E'] + combined_df['U']

print(f'Combined records: {len(combined_df)}')

In [ ]:
# District-level aggregation
district_df = combined_df.groupby(['state', 'district']).agg(
    total_months=('month', 'count'),
    active_months=('T', lambda x: (x > 0).sum()),
    total_E=('E', 'sum'), total_DU=('DU', 'sum'), total_BU=('BU', 'sum'),
    total_U=('U', 'sum'), total_T=('T', 'sum'),
    avg_monthly_enrolment=('E', 'mean'),
    monthly_volatility=('T', lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
    peak_load_ratio=('T', lambda x: x.max() / x.mean() if x.mean() > 0 else 0),
    sum_age_0_5=('age_0_5', 'sum'), sum_age_5_17=('age_5_17', 'sum')
).reset_index()

district_df['zero_months'] = district_df['total_months'] - district_df['active_months']
district_df['activity_ratio'] = district_df['active_months'] / district_df['total_months']
district_df['zero_month_ratio'] = district_df['zero_months'] / district_df['total_months']
district_df['biometric_burden'] = (district_df['total_BU'] / (district_df['total_BU'] + district_df['total_DU'])).fillna(0)
district_df['update_dominant'] = np.where(district_df['total_U'] > district_df['total_E'], 1, 0)
district_df['enrollment_update_balance'] = (district_df['total_E'] / (district_df['total_E'] + district_df['total_U'])).fillna(0)

print(f'Districts computed: {len(district_df)}')

## 4. DEI Score Calculation

In [ ]:
def normalize(x):
    min_val, max_val = x.min(), x.max()
    if max_val == min_val:
        return pd.Series([0.5] * len(x), index=x.index)
    return (x - min_val) / (max_val - min_val)

def inverse_normalize(x):
    return 1 - normalize(x)

scores_df = district_df.copy()

# DEI Components
scores_df['access'] = (scores_df['activity_ratio'] + normalize(scores_df['avg_monthly_enrolment'])) / 2
scores_df['responsiveness'] = normalize(scores_df['total_U'] / scores_df['total_T'])
scores_df['inclusion'] = normalize((scores_df['sum_age_0_5'] + scores_df['sum_age_5_17']) / scores_df['total_E'])
scores_df['stability'] = (inverse_normalize(scores_df['monthly_volatility']) + inverse_normalize(scores_df['peak_load_ratio'])) / 2
scores_df['visibility'] = scores_df['activity_ratio']

# Final scores
scores_df['DEI'] = (scores_df['access'] + scores_df['responsiveness'] + scores_df['inclusion'] + scores_df['stability'] + scores_df['visibility']) / 5
scores_df['ASS'] = (inverse_normalize(scores_df['activity_ratio']) + inverse_normalize(scores_df['avg_monthly_enrolment'])) / 2
scores_df['UBS'] = (normalize(scores_df['biometric_burden']) + normalize(scores_df['update_dominant'])) / 2
scores_df['SRS'] = (normalize(scores_df['monthly_volatility']) + normalize(scores_df['zero_month_ratio'])) / 2

print('DEI calculated!')
scores_df[['district', 'DEI', 'ASS', 'UBS', 'SRS']].sort_values('DEI', ascending=False)

In [ ]:
# Save outputs
scores_df.to_csv(os.path.join(BASE_PATH, 'telangana_district_analysis.csv'), index=False)
scores_df[['state', 'district', 'DEI', 'ASS', 'UBS', 'SRS']].to_csv(
    os.path.join(BASE_PATH, 'telangana_district_final_scores.csv'), index=False)
print('✅ Saved!')